In [8]:
class ExtractMeasurements:
    def __init__(self,c8y,source):
        '''
        ::PARAMETERS::
        c8y -> instância 'CumulocityApi' da lin c8y-api
        source -> id do device dentro da instância do Cumulocity IoT
        ::DESCRIPTION::
        inicia os dois objetos para utilizarmos nas demais funções
        as variaveis de timedelta são iniciadas aqui pois elas irão controlar o periodo da requisição na API

        '''
        
        from datetime import timedelta
        import pandas as pd

        #instância necessárias para realizar as operaçoes na API
        self.c8y = c8y
        self.source = source

        self.delta_twohours = timedelta(hours=2) # 2 hours older
        self.delta_onehour = timedelta(hours=1) # 1 hour older


    #classe para extrair de devices novos
    def extract_new_devices(self) -> list:
        '''
        ::PARAMETERS::
            Não possui.
        ::DESCRIPTION::
            há diferentes devices com formas de aquisição de dados diferentes pois a forma de subir para a nuvem do Cumulocity IoT foi diferente.
            Esse objeto contempla a extração de dados de devices que foram alocados no Cumulocity via conector do eWon. Tal forma de subir os dados é tendência por sua simplicidade e alta capacidade de controle de time interval.
        ::RETURN::
            retorna uma lista com dois dataframes.
            o df[0] é o dataframe com dados de UMA hora atrás
            o df[1] é o dataframe com dados de DUAS hora atrás
        '''
        def tratamento(_hour_measu):
            '''
            ::PARAMETER:: 
                _hour_measu -> generator gerado por CumulocityApi.measurements.select()
            :::DESCRIPTION::
                formata os dados requisitados num dataframe inserindo para cada dado de nome novo uma nova coluna no dataframe
                Alguns dados podem ter o intervalo de aquisição diferente, para isso, inserimos a var 'comp_min' para equilibrar os arrays e inserir num dataframe de linhas iguais.
            ::RETURN:::
                retorna o dataframe dinamicamente formatado
            '''
            df = {}
            for i,v in enumerate(_hour_measu):
                fragments = v.fragments # caracteristicas do dados puxado. Daí tiramos as propriedes que queremos
                col_name = list(fragments.keys())[0] # nome da nossa coluna no df
                aux = list(fragments[col_name].keys())[0] # parametro utilizado para entrar no aninhamento
                value = [list(fragments[col_name][aux].values())[1]] #valor do parametro atual -> inserimos ele numa lista para poder appendar no script abaixo

                # no primeiro loop, precisamos definir as colunas do dataframe, nos próximos basta appendarmos.
                try:
                    df[col_name].append(value[0])
                except KeyError as error:
                    df[col_name] = value

            comp_min = min(map(len,df.values())) # alguns parametros possuem time interval de envio de dados diferentes. Isso faz com que seja necessario equilibrarmos ele
                #limitando as linhas do dataframe
            for k,v in df.items():
                    df[k] = v[:comp_min]
                
            df = pd.DataFrame(df)
            return df
        
        #measurements de uma hora atrás
        one_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_onehour)
        #measurements de duas horas atras
        two_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_twohours, min_age=self.delta_onehour)
        
        # retorna uma lista com os dfs de uma hora atrás e duas horas atrás da mesma source
        return [tratamento(one_hour_measu),tratamento(two_hour_measu)]


    def extract_old_devices(self):
        def tratamento(_hour_measu):
            df = {}
            for i,v in enumerate(_hour_measu):
                fragments = v.fragments.values() # taking the actual parameter
                for f in fragments:
                    col_name = list(f.keys())[0] # setting 
                    value = [list(f[col_name].values())[1]]
                    
                    try:
                        df[col_name].append(value[0])
                    except KeyError as error:
                        df[col_name] = value

            comp_min = min(map(len,df.values()))
            for k,v in df.items():
                df[k] = v[:comp_min]
            df = pd.DataFrame(df)

            return df

        #measurements de uma hora atrás
        one_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_onehour)
        #measurements de duas horas atras
        two_hour_measu = self.c8y.measurements.select(source=self.source,max_age=self.delta_twohours, min_age=self.delta_onehour) 

        return [tratamento(one_hour_measu),tratamento(two_hour_measu)]        
    
        
        

# TESTE DA CLASSE

In [11]:
# libs necessárias
from c8y_api import CumulocityApi
from datetime import timedelta
import pandas as pd 

import json
import os

config_path = os.path.join(os.getcwd(), '..','..','..','','configs','config.json')
with open(config_path) as config_file:
    login = json.load(config_file)

# instâncias gloabais
c8y = CumulocityApi(
    base_url= login['url'],
    tenant_id= login['tenant'],
    username= login['username'],
    password= login['password'],
)

#devices de teste
# 621151045972 -> unilever || 1069740447 -> serbom || 611284292311 -> garoto cag 09 || 481304022994 -> garoto cag 10 || 891204422984 -> Fini Secomea || 581244909717 -> SODEBO
sources = ['621151045972', '1069740447', '611284292311','481304022994','891204422984', '581244909717']


In [12]:
extract = ExtractMeasurements(c8y=c8y,source=sources[4])

In [14]:
extract.extract_new_devices()

IndexError: list index out of range